In [ ]:
from dataloader import load_dataset
import numpy as np
from iron import phi
import numpy as np
from bayes_opt import BayesianOptimization
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from dataloader import load_dataset
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
import csv
from collections import OrderedDict
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import MinMaxScaler
import os


In [9]:
##################   FOR GETTING FIRST TIME RESULTS   ##################
def get_algorithmic_recourse_results(name, model, step, is_global, multiplier):
    '''
    Args:
    - name (str)            : dataset name
    - model (str)           : model to run
    - step (int)            : data point frequency usae
    - is_global (bool)      : if we are optimizing for the maximum we can get or a specific target
    - multiplier (float)    : target multiplier, only if we are looking for a specific taret

    update csv with new structure
    '''
    # PRE PROCESSING DATA AND GETTING DATA
    X_train, y_train, X_test, y_test, features, ph, phis, reg = pre_process_data(name, model)

    print('starting baysopt')
    # getting results from bayesian optimization
    abs_results, pdiffs_abs_y, params, pred_y_abs = run_all_abs_y(features, reg, X_test, y_test, step, is_global, multiplier)
    rel_results, pdiffs_abs_rel, params_rel, pred_y_rel = run_all_abs_rel(features, reg, X_test, y_train, y_test, step, phis, ph, is_global, multiplier)

    # getting the iterations of each result (the value predicted from each iteration of the bayesopt model, so 55 for each)
    absolute_iterations = get_iterations(pred_y_abs)
    relevance_iterations = get_iterations(pred_y_rel)

    # p1, p2, f1, f2, r1, r2 = get_paths(name, model)
    pdiffs_abs_file, pdiffs_rel_file, f_abs_file, f_rel_file, r_abs_file, r_rel_file = get_paths(name, model, is_global, multiplier)

    # saves data into csv
    all_dfs_to_csvs(pdiffs_abs_file, pdiffs_rel_file, f_abs_file, f_rel_file, r_abs_file, r_rel_file, pdiffs_abs_y, pdiffs_abs_rel, params, params_rel, absolute_iterations, relevance_iterations)

    # retrieves differences and features from the files we created earlier
    pdiffs_abs = read_file(pdiffs_abs_file)
    pdiffs_rel = read_file(pdiffs_rel_file)
    features_abs = read_file(f_abs_file)
    features_rel = read_file(f_rel_file)

    # changes features to ensure order?
    new_f_abs = change_dict(read_dict_file(f_abs_file), features)
    new_f_rel = change_dict(read_dict_file(f_rel_file), features)
    features_abs = df_to_csv(new_f_abs, f_abs_file)
    features_rel = df_to_csv(new_f_rel, f_rel_file)
    
    # features_abs, features_rel = convert_new_df(new_f_abs, new_f_rel, f_abs_file, f_rel_file)
    features_abs = read_file(f_abs_file)
    features_rel = read_file(f_rel_file)

################## 1. DATA PRE PROCESSING ##################

def pre_process_data(name, model):
    '''
    Args:
    - name (str)            : dataset name
    - model (str)           : model to run
    - step (int)            : data point frequency usae

    returns X/y test/train data, list of feature names, global relevance functions, and regressor
    '''
    X_train, y_train, X_test, y_test = get_data(name)

    # FOR ONE OF THE DATASETS, THIS RENAMES COLUMNS TO BE USABLE BY BAYESIAN OPTIMIZATION
    X_train = X_train.rename(columns = {"fixed.acidity": "fixed_acidity",	"volatile.acidity": "volatile_acidity",	"citric.acid": "citric_acid", "residual.sugar": "residual_sugar", "free.sulfur.dioxide": "free_sulfur_dioxide", "total.sulfur.dioxide": "total_sulfur_dioxide"})
    X_test = X_test.rename(columns = {"fixed.acidity": "fixed_acidity",	"volatile.acidity": "volatile_acidity",	"citric.acid": "citric_acid", "residual.sugar": "residual_sugar", "free.sulfur.dioxide": "free_sulfur_dioxide", "total.sulfur.dioxide": "total_sulfur_dioxide"})

    print('received data')
    features = list(X_test.columns.values)
    print(", ".join(X_train.columns))
    ph, phis = get_phis_global(y_train, y_test)
    print("received phi/phis")
    reg = create_regressor(X_train, y_train, model)
    print('received regressor')

    return X_train, y_train, X_test, y_test, features, ph, phis, reg

def get_data(name):
    '''
    Args:
    - name (str)            : dataset name

    pre-process data (scale, train test split)
    return X_train, X_test, y_train, Y_test
    '''
    # get train test split
    if file_is_of_dataset(name): X_train, y_train, X_test, y_test = load_dataset(name)
    else: X_train, y_train, X_test, y_test = manually_label_categorical_data(name)

    # one hot encode
    X_train = encode_categorical_columns(X_train)
    X_test = encode_categorical_columns(X_test)

    # fit scaler to X_train
    scaler = MinMaxScaler()
    scaler.fit(X_train)

    # fit transform X_train and X_test
    scaled_X_train = scaler.transform(X_train)
    scaled_X_test = scaler.transform(X_test)

    X_train = pd.DataFrame(scaled_X_train, index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(scaled_X_test, index=X_test.index, columns=X_test.columns)

    # print("X TRAIN AFTER SCALED")
    # print(X_train)

    y_test = y_test.astype(float)
    y_train = y_train.astype(float)

    return X_train, y_train, X_test, y_test

def encode_categorical_columns(df):
    '''
    Args:
    - df (df)            : dataframe to encode

    encodes categorical columns through on hot encoder
    return encoded df
    '''
    
    # get categorical column names
    categorical_columns = df.select_dtypes(include='category').columns.tolist()

    # initialize OneHotEncoder
    # encoder = OneHotEncoder(sparse_output=False)  # Setting drop='first' to avoid dummy variable trap # this is deprecated
    encoder = OneHotEncoder(sparse=False)  # Setting drop='first' to avoid dummy variable trap

    # Fit and transform on categorical columns
    encoded_data = encoder.fit_transform(df[categorical_columns])

    # Create column names for new features
    new_columns = encoder.get_feature_names_out(categorical_columns)

    encoded_df = pd.DataFrame(encoded_data, columns=new_columns, index=df.index)

    # Concatenate with original DataFrame, dropping original categorical columns
    df_encoded = pd.concat([df.drop(columns=categorical_columns), encoded_df], axis=1)

    return df_encoded

def manually_label_categorical_data(name):
    '''
    Args:
    - name (str) : name of dataset to use
    
    for manually one hot encoding if file is not part of the provided datasets 
    
    returns x/y train/test split encoded data
    '''
    p = path + f'all_datasets/{name}.csv'
    df = pd.read_csv(p)

    y = df['charges']  # Get the target column
    X = df.drop(columns=['charges'])  # Drop the targetcolumn to get features

    # set columns as categorical
    X['sex'] = X['sex'].astype('category')
    X['children'] = X['children'].astype('category')
    X['smoker'] = X['smoker'].astype('category')
    X['region'] = X['region'].astype('category')

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    y_train = y_train.values.astype(float)
    y_test = y_test.values.astype(float)

    print(X_train.columns)

    return X_train, y_train, X_test, y_test

def get_phis_local(y_train, y_test, y_min, y_max, y_orig, y_target):
    '''
    Args:
    - y_train (array)     : y train
    - y_test (array)      : y test
    - y_min (float)       : minimum y
    - y_max (float)       : max y
    - y_orig (float)      : original y
    - y_target (float)    : target y we optimize for (relevance = 1)
    Returns, ph, phis


    Returns relevance function for a specific point we want to get
    and the according phis for the test dataset
    '''
    pts = [y_min, 0, 0,
    y_orig, 0.5, 0,
    y_target, 1, 0,
    y_max, 0, 0]

    if y_orig == y_min:
        pts = [y_orig, 0.5, 0,
        y_target, 1, 0,
        y_max, 0, 0]
    if y_orig == y_max:
        pts = [y_min, 0, 0,
        y_orig, 0.5, 0,
        y_target, 1, 0]
    if y_orig == 0:
        pts = [y_min, 0.5, 0,
        y_orig, 1, 0,
        y_max, 0, 0]
    if y_orig == y_min and y_orig == 0:
        pts = [y_target, 1, 0,
        y_max, 0, 0]
    if y_orig == y_max and y_orig == 0:
        pts = [y_min, 0.5, 0,
        y_orig, 1, 0]

    # print(f"points: {pts}")

    npts = int(len(pts) / 3)


    # positive change
    control_pts = {
    "control_pts": pts,
    "npts": npts,
    }
    # print(f"ctrl_pts : {control_pts}")

    ph = phi.phi_control(y_train, control_pts=control_pts, method="range")
    phis = phi.phi(pd.Series(y_test), phi_parms=ph)

    # print(f'ph: {ph}')
    # print(f'phis: {phis}')

    return ph, phis

def get_phis_global(y_train, y_test):
    '''
    Args:
    - y_train (array)     : y train
    - y_test (array)      : y test

    returns global relevance functions (towards right extreme)
    '''
    # solve something at the dataset level
    ph = phi.phi_control(pd.Series(y_train), extr_type="high")
    # relevance (phis) on test
    phis = phi.phi(pd.Series(y_test), phi_parms=ph)

    return ph, phis

def create_regressor(X_train, y_train, model):
    '''
    Args:
    - X_train (array)       : X train data
    - y_train (array)       : y train data
    - model (str)           : model name we want to run

    Returns regression model based on the data and type
    '''

    if model == "rf": reg = RandomForestRegressor(n_estimators=500, random_state = 0) # n = 500
    else: reg = LGBMRegressor(n_estimators=500, random_state = 0)

    reg.fit(X_train.values, y_train)

    return reg


################## 2. RUNNING BAYESOPT ##################
def run_all_abs_y(features, reg, X_test, y_test, step, is_global, multiplier):
    '''
    Args:
    - features (array)      : list of features
    - reg (model)           : regression model
    - X_test (array)        : X test data
    - y_test (array)        : y test data
    - step (int)            : data point frequency
    - is_global (bool)      : chooses if we want to run entire data or not
    - multiplier (float)    : target we optimize for
    
    NOTE: MUST REPLACE THE FEATURES IN THE BLACK_BOX FUNCTION AND X 2D ARRAY WITH FEATURES YOU WANT TO RUN

    runs bayesian optimization for algorithmic recourse

    returns abs_results, pdiffs_abs_y, params, pred_ys
    '''
    print("start abs bayesopt")
    print(features)

    pdiffs_abs_y = []
    pred_ys = []
    params = []
    # pred_y_rel = []
    # rel = []
    abs_results = []

    base = []
    pbounds = {} # 5%

    p_mult = 1 + multiplier
    n_mult = 1 - multiplier


    # FOR CREATING PBOUNDS + STARTING X (base)
    # for each row/feature in our dataset
    for i in range(0, len(X_test), step):
        predicted_ys = []
        y_orig = y_test[i] # get true y


        if not is_global:
            if y_test[i] < 0: mult = n_mult
            else: mult = p_mult
            y_target = y_test[i] * mult # target val we want to reach

            y_diff = abs(y_target - y_test[i]) # distance(difference) from that target value

        # creating pbounds based on mean of features
        for f in features:
            x = X_test.iloc[i][f]
            base.append(x)
            if is_global: sd = 0.05 # 
            else: sd = 1.0          #
            pbounds[f] = (min(x - (sd * x), x + (sd * x)), max(x - (sd * x), x + (sd * x)))

        # optimizes for maximum y difference
        def black_box_function_abs_y(



age, bmi, sex_female, sex_male, children_0, children_1, children_2, children_3, children_4, children_5, smoker_no, smoker_yes, region_northeast, region_northwest, region_southeast, region_southwest


        ):

            x = [[



age, bmi, sex_female, sex_male, children_0, children_1, children_2, children_3, children_4, children_5, smoker_no, smoker_yes, region_northeast, region_northwest, region_southeast, region_southwest


            ]]

            y1 = reg.predict(x)     # predicted y value
            predicted_ys.append(y1[0])

            if is_global:
                if y_orig != 0: pdiff = ((y1[0] - y_orig) / abs(y_orig)) * 100  # difference between original and predicted y
                else: pdiff = 0
            else:
                if y_diff == 0: pdiff = 0
                else:
                    pdiff = -abs(y1[0] - y_target) / y_diff

            return pdiff

        ########### CREATING BAYESIAN OPTIMIZER ###########
        optimizer_abs_y = BayesianOptimization(
            f=black_box_function_abs_y,  # f(x)
            pbounds=pbounds,  # parameter bounds
            random_state=1,
            allow_duplicate_points=True,
            verbose=0,
        )

        ########### MAXIMIZING (finding optimums) ###########
        optimizer_abs_y.maximize(
            init_points=5,  # num of random steps to perform --> can help diversify explore space
            n_iter=50,  # number of iterations/steps of baye opt --> more = more accurate optimum
        )


        opt = optimizer_abs_y.max['params']                 # features for the best case
        pdiffs_abs_y.append(optimizer_abs_y.max['target'])  # y difference for best case
        params.append(opt)
        abs_results.append(optimizer_abs_y.res)

        # iteration = []
        # for it in optimizer_abs_y.res:
        #     # params for the best case
        #     param = []
        #     for val in it['params'].values(): param.append(val)
        #     # optimized prediction for this case
        #     pred = reg.predict([param])[0]
        #     iteration.append(pred)

        # abs_results.append(iteration)

        pred_ys.append(predicted_ys)

    print("end abs bayesopt")


    return abs_results, pdiffs_abs_y, params, pred_ys

def run_all_abs_rel(features, reg, X_test, y_train, y_test, step, phis, ph, is_global, multiplier):
    '''
    Args:
    - features (array)      : list of features
    - reg (model)           : regression model
    - X_test (array)        : X test data
    - y_test (array)        : y test data
    - step (int)            : data point frequency
    - phis (array)          : relevance of each y
    - ph (ph)               : relevance function
    - is_global (bool)      : chooses if we want to run entire data or not
    - multiplier (float)    : target we optimize for
    
    NOTE: MUST REPLACE THE FEATURES IN THE BLACK_BOX FUNCTION AND X 2D ARRAY WITH FEATURES YOU WANT TO RUN

    runs bayesian optimization based on the target

    returns rel_results, pdiffs_abs_rel, params_rel, pred_ys
    
    '''
    print("start rel bayesopt")

    pdiffs_abs_rel = []
    params_rel = []
    pred_ys = []
    rel_results = []

    base = []
    pbounds = {}

    y_min = min(y_test)
    y_max = max(y_test)


    p_mult = 1 + multiplier
    n_mult = 1 - multiplier

    # CREATES PBOUNDS AND BASE X VALUES
    for i in range(0, len(X_test), step):
        predicted_ys = []

        y_rel_global = phis[i]

        # for a local/specific point we want to get
        if not is_global:
            if y_test[i] < 0: mult = n_mult
            else: mult = p_mult
            # TARGET Y WE WANT TO GET
            y_target = y_test[i] * mult
            # relevance of all y according to the specific relevance function we created
            ph, phis = get_phis_local(y_train, y_test, y_min, y_max, y_test[i], y_target)
            # true relevances for all y_test w/ our specific relevance function created for every y_test
            # y_rel_diff = abs(1 - phis[i])
            # print(f"y_rel_diff = {y_rel_diff}, y_rel_orig = {phis[i]}, y_rel_target = {phi.phi(pd.Series(y_target), phi_parms=ph)}")

        y_rel = phis[i]                             # true relevance of the y
            # relevance difference between target relevance and



        # creating pbounds based on mean of features
        for f in features:
            x = X_test.iloc[i][f]
            base.append(x)
            if is_global: sd = 0.05
            else: sd  = 1.0
            pbounds[f] = (min(x - (sd * x), x + (sd * x)), max(x - (sd * x), x + (sd * x)))


        def black_box_function_abs_rel(


age, bmi, sex_female, sex_male, children_0, children_1, children_2, children_3, children_4, children_5, smoker_no, smoker_yes, region_northeast, region_northwest, region_southeast, region_southwest




):
            # when generating x, we will have a list of vars with all the numeric and categorical
            x = [[

age, bmi, sex_female, sex_male, children_0, children_1, children_2, children_3, children_4, children_5, smoker_no, smoker_yes, region_northeast, region_northwest, region_southeast, region_southwest


]]

            y1 = reg.predict(x)                                         # predicted y
            y_rel_pred = phi.phi(pd.Series(y1[0]), phi_parms=ph)        # relevance of the predicted y
            if is_global:
                pdiff = y_rel_pred[0]
                # if y_rel != 0: pdiff = ((y_rel_pred[0] - y_rel)/abs(y_rel)) * 100                 # optimize for better positive relevance change
                # else: pdiff = 0
            # closer our predicted rel is, the better
            else: pdiff = y_rel_pred[0]                                 # optimize for the better y

            predicted_ys.append(y1[0])

            return pdiff

        ########### CREATING BAYESIAN OPTIMIZER ###########
        optimizer_abs_rel = BayesianOptimization(
            f=black_box_function_abs_rel,  # f(x)
            pbounds=pbounds,  # parameter bounds
            random_state=1,
            allow_duplicate_points=True,
            verbose=0,
        )

        ########### MAXIMIZING (finding optimums) ###########
        optimizer_abs_rel.maximize(
            init_points=5,  # num of random steps to perform --> can help diversify explore space
            n_iter=50,  # number of iterations/steps of baye opt --> more = more accurate optimum
        )

        opt = optimizer_abs_rel.max['params']                           # features for the best case
        pdiffs_abs_rel.append(optimizer_abs_rel.max['target'])          # relevance difference for the best case
        params_rel.append(opt)
        rel_results.append(optimizer_abs_rel.res)


        # iteration = []
        # for it in optimizer_abs_rel.res:
        #     # params for the best case
        #     param = []
        #     for val in it['params'].values(): param.append(val)
        #     # optimized prediction for this case
        #     pred = reg.predict([param])[0]
        #     iteration.append(pred)

        # rel_results.append(iteration)

        pred_ys.append(predicted_ys)



    print("end rel bayesopt")

    return rel_results, pdiffs_abs_rel, params_rel, pred_ys

################## 3. ANALYZING DATA AND STORING THEM ##################

# get the iterations from each bayesopt run
def get_iterations(results):
    return results

def get_iterations_till_target(y_test, params, iterations, reg, step, multiplier):
    '''
    Args:
    - y_test (array)        : y test data
    - params                : parameters for the model
    - iterations            : iterations of the model
    - reg (model)           : regression model
    - step (int)            : data point frequency
    - multiplier (float)    : target we optimize for

    finds the number of iterations to get the best value

    returns iterations_till_target, err
    '''
    
    iterations_till_target = []
    p_mult = 1 + multiplier
    n_mult = 1 - multiplier

    # loops over iterations (it = array of iterations of one case)
    for i, it in enumerate(iterations):
        # getting optimized (predicted) y
        if i == 0: continue # skip header
        i -= 1
        x_param = [params[i]]
        y_pred = reg.predict(x_param)[0]

        # getting target y
        if y_test[i * step] < 0: mult = n_mult
        else: mult = p_mult
        y_target = y_test[i * step] * mult

        added = False

        # index, predicted y for that iteration (1-50)
        for j, pred in enumerate(it, 1):
            if added: break
            if pred != y_pred: continue

            # if this is the case we got the prediction, add to list
            # iterations_till_target.append((j, pred, y_target))
            iterations_till_target.append(j)
            added = True


        if not added: iterations_till_target.append(-1000000)

    if -1000000 in iterations_till_target: err = 0
    else: err = 1
    return iterations_till_target, err

def get_paths(name, model, is_global, multiplier):
    '''
    Args:
    - name (str)            : dataset name
    - model (str)           : model to run
    - is_global (bool)      : if we are optimizing for the maximum we can get or a specific target
    - multiplier (float)    : target multiplier

    get paths to store the results in
    '''
    if is_global:
        # suffix = "_global"
        suffix = ""
        prefix = "global_"

    else:
        suffix = ""
        prefix = "specific-" + str(multiplier) + "-"


    # path = f"/Users/johnkim/Developer/algorithmic recourse/all_datasets/"
    # poop
    pdiff_path = path + prefix + "results/pdiffs/"
    feat_path = path + prefix + "results/features/"
    iter_path = path + prefix + "results/iterations/"
    if not os.path.exists(pdiff_path): os.makedirs(pdiff_path)
    if not os.path.exists(feat_path): os.makedirs(feat_path)
    if not os.path.exists(iter_path): os.makedirs(iter_path)
        
    p1 = pdiff_path + name + "_" + model + suffix + "_abs.csv"
    p2 = pdiff_path + name + "_" + model + suffix + "_rel.csv"
    f1 = feat_path + name + "_" + model + suffix + "_abs.csv"
    f2 = feat_path + name + "_" + model + suffix + "_rel.csv"
    r1 = iter_path + name + "_" + model + suffix + "_abs.csv"
    r2 = iter_path + name + "_" + model + suffix + "_rel.csv"

        # p1 = "/Users/johnkim/Developer/algorithmic recourse/" + prefix + "csv/pdiffs_" + name + "_" + model + suffix + "_abs_new.csv"
        # p2 = "/Users/johnkim/Developer/algorithmic recourse/" + prefix + "csv/pdiffs_" + name + "_" + model + suffix + "_rel_new.csv"
        # f1 = "/Users/johnkim/Developer/algorithmic recourse/" + prefix + "csv/f_" + name + "_" + model + suffix + "_abs_new.csv"
        # f2 = "/Users/johnkim/Developer/algorithmic recourse/" + prefix + "csv/f_" + name + "_" + model + suffix + "_rel_new.csv"
        # r1 = "/Users/johnkim/Developer/algorithmic recourse/" + prefix + "csv/r_" + name + "_" + model + suffix + "_abs.csv"
        # r2 = "/Users/johnkim/Developer/algorithmic recourse/" + prefix + "csv/r_" + name + "_" + model + suffix + "_rel.csv"


    return p1, p2, f1, f2, r1, r2

# save dataframe into a csv
def df_to_csv(dataset, path):
    '''
    Args:
    - dataset (df)          : dataframe 
    - path (str)            : path to data

    store df into a csv at the path
    '''
    df = pd.DataFrame(dataset)
    df.to_csv(path)

    return df

# saves dfs into csvs
def all_dfs_to_csvs(p1, p2, f1, f2, r1, r2, pdiffs_abs_y, pdiffs_abs_rel, params, params_rel, absolute_iterations, relevance_iterations):
    '''
    Args:
    - p1, p2, f1, f2, r1, r2 (str)  : path names
    - pdiffs_abs_y, pdiffs_abs_rel, params, params_rel, absolute_iterations, relevance_iterations   : dataframes of our results

    save all results to a csv
    '''
   # return df_to_csv(p1, pdiffs_abs_y), df_to_csv(p2, pdiffs_abs_rel), df_to_csv(f1, params), df_to_csv(f2, params_rel), df_to_csv(r1, absolute_iterations), df_to_csv(r2, relevance_iterations)
    return df_to_csv(pdiffs_abs_y, p1), df_to_csv(pdiffs_abs_rel, p2), df_to_csv(params, f1), df_to_csv(params_rel, f2), df_to_csv(absolute_iterations, r1), df_to_csv(relevance_iterations, r2)

def read_file(file):
    '''
    Args:
    - file (file)            : dataset name

    read file
    '''
    arr = []
    with open(file, newline='') as f:
        reader = csv.reader(f)
        for lines in reader:
            if (len(lines) == 2): arr.append(float(lines[1]))
            else:
                try:
                    arr.append(list(np.float64(lines[1:])))
                except ValueError:
                    pass


        if arr[0] == 0: arr.pop(0)


    return arr

def read_dict_file(file):
    '''
    Args:
    - file (str)            : filename

    open and read file dict
    '''
    arr = []
    with open(file, 'r') as f:
        dict_reader = csv.DictReader(f)
        for line in dict_reader:
            line.pop('', None)
            arr.append(line)

    return arr

def read_file_iterations(file):
    '''
    Args:
    - file (file)            : filename

    read the iterations from the file
    '''
    results = {}
    with open(file, 'r') as f:
        reader = csv.reader(f)
        for i, lines in enumerate(reader):
            if lines[0] == '': continue

            lines = lines[1:]

            results[i] = [float(i) for i in lines]

            # lines[1:]

    # print(results)
    # print(len(results[0]))
    return results

def change_dict(d, features):
    '''
    Args:
    - d (dictionary)          : dictionary
    - features (array)        : features of the dataset


    get paths to store the results in
    '''
    arr = []
    for row in d:
        r = OrderedDict()
        for f in features:
            r[f] = row[f]
        arr.append(r)

    return arr

def convert_new_df(d1, d2, path1, path2):
    '''
    Args:
    - d1 (df)           : dataframe
    - d2 (df)           : dataframe
    - path1 (str)       : path of file
    - path2 (str)       : path of file

    convert to new dataframe
    '''

    df1 = pd.DataFrame(d1)
    df1.to_csv(path1)

    df2 = pd.DataFrame(d2)
    df2.to_csv(path2)

    return df1, df2

def target_calculate_relevance_difference(params, y_train, y_test, pdiffs, reg, step, multiplier):
    '''
    Args:
    - params (list[dict])   : parameters of the optimized y we found
    - y_train (array)     : y train
    - y_test (array)      : y test
    - pdiffs (list[float])  : list of differences in true - predicted
    - reg (model)           : regression model
    - step (int)            : iterate every step size
    - multiplier (int)      : multiplier target

    Returns list[float] of differences between true and predicted y relevance for given optimizations
    '''
    difference_of_relevance = []
    y_min = min(y_test)
    y_max = max(y_test)


    p_mult = 1 + multiplier
    n_mult = 1 - multiplier

    # for however cases we did
    for i in range(len(pdiffs)):
        # have to find the specific phi function for each one to predict
        if y_test[i * step] < 0: mult = n_mult
        else: mult = p_mult

        y_target = y_test[i * step] * mult
        ph, phis = get_phis_local(y_train, y_test, y_min, y_max, y_test[i * step], y_target)
        # parameters for optimized y
        x_param = [params[i]]
        # optimal y we found
        y_pred = reg.predict(x_param)[0]
        # rel of optimal y
        y_pred_rel = phi.phi(pd.Series(y_pred), phi_parms=ph)
        # rel of true y
        # y_true_rel = phis[i * step]
        # y_rel_diff = abs(1 - phis[i * step])
        # print(f"original y = {y_test[i * step]}, original rel = {phis[i * step]}, optimal y = {y_pred}, optimal rel = {y_pred_rel[0]}")

        relevance_difference = y_pred_rel
        # relevance_difference = -abs(y_pred_rel[0] - y_true_rel) / y_rel_diff
        # print(f"rel diff = {abs(y_pred_rel[0] - y_true_rel)}")
        # print(f"rel pdiff = {relevance_difference}")

        # closer our predicted rel is, the better
        # relevance_difference = -abs(y_pred_rel - y_true_rel)

        difference_of_relevance.append(relevance_difference)

    return difference_of_relevance

def calculate_relevance_difference(params, pdiffs, reg, phis, ph, step):
    '''
    Args:
    - params (list[dict])   : parameters of the optimized y we found
    - pdiffs (list[float])  : list of differences in true - predicted
    - reg (model)           : regression model
    - phis (list[float])    : list of actual relevance for y_test
    - ph (relevacnce func)  : relevance function used to create phis
    - step (int)            : iterate every step size

    Returns list[float] of differences between true and predicted y relevance for given optimizations
    '''
    difference_of_relevance = []

    # for however cases we did
    for i in range(len(pdiffs)):
        # parameters for optimized y
        x_param = [params[i]]
        # optimal y we found
        y_pred = reg.predict(x_param)[0]
        # rel of optimal y
        y_pred_rel = phi.phi(pd.Series(y_pred), phi_parms=ph)
        # rel of true y
        y_true_rel = phis[i * step]

        relevance_difference = y_pred_rel
        difference_of_relevance.append(relevance_difference[0])

    return difference_of_relevance

def target_calculate_y_difference(y_test, params, pdiffs, reg, step, multiplier):
    '''
    Args:
    - y_test (list)         : list of all y values in test dataset
    - params (list[dict])   : parameters of the optimized y we found
    - pdiffs (list[float])  : list of differences in true - predicted
    - reg (model)           : regression model
    - step (int)            : iterate every step size
    - multiplier (int)      : multiplier target

    Returns list[float] of differences between true and predicted y values for given optimizations
    '''
    difference_of_y = []

    p_mult = 1 + multiplier
    n_mult = 1 - multiplier

    # this is how many y's we ran for
    for i in range(len(pdiffs)):
        if y_test[i * step] < 0: mult = n_mult
        else: mult = p_mult

        # the optimized y parameter
        x_param = [params[i]]

        # predict the optimal y we found
        y_pred = reg.predict(x_param)[0]
        # get the original y
        y_target = y_test[i * step] * mult
        # difference of y
        y_diff = abs(y_target - y_test[i * step])
        if y_diff == 0: abs_diff = 0
        else: abs_diff = -abs(y_pred - y_target) / y_diff
        # ^^ should be what we use to optimize
        # after we need to take the optimized cases and convert to actual diff

        '''
        ( 1 - 1.2 ) / 0.2       = -100%
        vs.
        (1.4 - 1.2 ) / 0.2      = 100%
        '''


        # print(f"optimal y: {y_pred}, original y: {y_test[i * step]}, target y: {y_target}, y_diff: {y_diff}")

        difference_of_y.append(abs_diff)


    # return
    return difference_of_y

def calculate_y_difference(y_test, params, pdiffs, reg, step):
    '''
    Args:
    - y_test (list)         : list of all y values in test dataset
    - params (list[dict])   : parameters of the optimized y we found
    - pdiffs (list[float])  : list of differences in true - predicted
    - reg (model)           : regression model
    - step (int)            : iterate every step size

    Returns list[float] of differences between true and predicted y values for given optimizations
    '''
    difference_of_y = []

    # this is how many y's we ran for
    for i in range(len(pdiffs)):
        # the optimized y parameter
        x_param = [params[i]]

        # predict the optimal y we found
        y_pred = reg.predict(x_param)[0]
        # get the original y
        y_orig = y_test[i * step]
        # get the difference between the two
        abs_diff = y_pred - y_orig
        if y_orig == 0: abs_diff = 0
        else: abs_diff = ((y_pred - y_orig) / abs(y_orig)) * 100

        difference_of_y.append(abs_diff)

    # return
    return difference_of_y

def target_calculate_y_percent_difference(y_test, params, pdiffs, reg, step, multiplier):
    '''
    Args:
    - y_test (list)         : list of all y values in test dataset
    - params (list[dict])   : parameters of the optimized y we found
    - pdiffs (list[float])  : list of differences in true - predicted
    - reg (model)           : regression model
    - step (int)            : iterate every step size
    - multiplier (int)      : multiplier target

    Returns list[float] of differences between true and predicted y values for given optimizations
    '''
    percent_diff = []

    p_mult = 1 + multiplier
    n_mult = 1 - multiplier

    # this is how many y's we ran for
    for i in range(len(pdiffs)):
        if y_test[i * step] < 0: mult = n_mult
        else: mult = p_mult
        # the optimized y parameter
        x_param = [params[i]]
        # predict the optimal y we found
        y_pred = reg.predict(x_param)[0]
        # get the original y
        y_target = y_test[i * step] * mult
        # difference of y
        y_diff = abs(y_target - y_test[i * step])
        if y_diff == 0: abs_diff = 0
        else: abs_diff = (y_pred - y_target) / y_diff
        # orig = 10, target = 20, y_pred = 7
        # 7 - 20 / 10 = -0.3
        # 19 - 20 / 10 = -0.1
        # 21 - 20 / 10 = 0.1
        percent_diff.append(abs_diff)

    return percent_diff

def calculate_distance(X_test, features_abs, features_rel, step):
    '''
    Args:
    - X_test (list)         : X test data
    - features_abs (list)   : features for rel diff
    - features_rel (list)   : features for y diff
    - step (int)            : iterate every step size

    calculates distance between original feature and predicted feature
    '''
    params_orig = []

    X_f = X_test.values.tolist()
    # X_f_scaled = (X_f-np.min(X_f))/(np.max(X_f)-np.min(X_f))

    # store original parameters
    for i in range(0, len(X_f), step):
        params_orig.append(X_f[i])

    # set the optimized parameters
    f1 = np.array(features_abs)
    f2 = np.array(features_rel)

    # find distances between the two
    distance_abs = euclidean_distances(params_orig, f1)
    distance_rel = euclidean_distances(params_orig, f2)

    d_abs = []
    d_rel = []

    # find the mean distance between them
    for d in distance_abs: d_abs.append(np.mean(d))
    for d in distance_rel: d_rel.append(np.mean(d))


    relative_diff = []
    squared_diff = []
    for i in range(len(d_rel)):
        relative_diff.append(((d_rel[i] - d_abs[i]) / d_abs[i]) * 100)
        squared_diff.append((d_rel[i] - d_abs[i]) * (d_rel[i] - d_abs[i]))

    return distance_abs, distance_rel, squared_diff
    # return relative_diff, squared_diff

#####################   FOR RETRIEVING DATA + CALCULATING STATISTICS   #####################

def process_results(names, models, file_intermediate, file_result, is_global, multiplier, step):
    '''
    Args:
    - name (str)            : dataset name
    - model (str)           : model to run
    - file_intermediate     : intermediate filename
    - file_result           : result filename
    - is_global (bool)      : if we are optimizing for the maximum we can get or a specific target
    - step (int)            : data point frequency usae
    - multiplier (float)    : target multiplier

    process all results
    '''
    # fields for csv results
    if is_global: fields = ['Name', 'Model', 'Optimize For', 'Y Mean', 'Y SD', 'Relevance Mean', 'Relevance SD', 'Distance', 'Squared Distance']
    else: fields = ['Name', 'Model', 'Optimize For', 'Y Mean', 'Y SD', 'Relevance Mean', 'Relevance SD', 'Iterations', 'Distance', 'Squared Distance']
    
    results = run_all_files(names, models, is_global, multiplier, step)
    convert_results_to_csv(file_intermediate, results)
    update_csv(fields, file_intermediate, file_result, is_global)

    print(f'Finished converting results!\n File is named: {file_result}')

def run_all_files(names, models, is_global, multiplier, step):
    '''
    Args:
    - names (list[str])     : names of datasets we want to run
    - models (list[str])    : models (rf or lgbm)
    - is_global (bool)      : if we are optimizing for the maximum we can get or a specific target
    - multiplier (float)    : target multiplier
    - step (int)            : data point frequency
    
    Runs 'run_file' on every file we have results for and returns in a list to convert into a csv
    '''
    results = []
    for name in names:
        print(f"Running {name}.")
        for model in models:
            print(f"Running {model}.")
            results.append(run_file(name, model, is_global, multiplier, step))
            print(f"Finisihed running {name} with {model}.")

    return results

def run_file(name, model, is_global, multiplier, step):
    '''
    Args:
    - name (str)    : dataset name
    - model (str)   : model name (rf or lgbm)
    - is_global (bool)      : if we are optimizing for the maximum we can get or a specific target
    - multiplier (float)    : target multiplier
    - step (int)            : data point frequency
    
    Run file --> get data from csv --> get statistics of interest --> return dictionary of the statistics
    '''
    X_train, y_train, X_test, y_test, ph, phis, reg, pdiffs_abs, pdiffs_rel, features_abs, features_rel, iterations_abs, iterations_rel, step = get_all_data(name, model, is_global, multiplier, step)

    print('getting diff')

    # getting relevance and y diff for the other data
    if is_global:
        difference_of_relevance_abs_y = calculate_relevance_difference(features_abs, pdiffs_abs, reg, phis, ph, step)
        difference_of_relevance_abs_rel_y = calculate_relevance_difference(features_rel, pdiffs_rel, reg, phis, ph, step)
        the_difference_of_y = calculate_y_difference(y_test, features_rel, pdiffs_rel, reg, step)
        diff_y = calculate_y_difference(y_test, features_abs, pdiffs_abs, reg, step) #IS SAME AS PDIFFS_ABS
    else:
        difference_of_relevance_abs_y = target_calculate_relevance_difference(features_abs, y_train, y_test, pdiffs_abs, reg, step, multiplier)
        difference_of_relevance_abs_rel_y = target_calculate_relevance_difference(features_rel, y_train, y_test, pdiffs_rel, reg, step, multiplier)
        the_difference_of_y = target_calculate_y_percent_difference(y_test, features_rel, pdiffs_rel, reg, step, multiplier)
        diff_y = target_calculate_y_percent_difference(y_test, features_abs, pdiffs_abs, reg, step, multiplier)
        # the_difference_of_y = target_calculate_y_difference(y_test, features_rel, pdiffs_rel, reg, step, multiplier)
        # diff_y = target_calculate_relevance_difference(features_abs, y_train, y_test, pdiffs_abs, reg, step, multiplier)



    # if pdiffs_abs != diff_y: raise Exception('pdiffs_abs != diff_y')
    # if pdiffs_rel != difference_of_relevance_abs_rel_y: raise Exception('pdiffs_rel != difference_of_relevance_abs')

    if pdiffs_abs != diff_y: print("pdiffs_abs != diff_y")
    else: print("pdiffs_abs EQUAL to diff_y")
    # print(pdiffs_abs)
    # print(diff_y)

    print()

    if pdiffs_rel != difference_of_relevance_abs_rel_y: print("pdiffs_rel != difference_of_relevance_abs_rel_y")
    else: print("pdiffs_rel EQUAL to difference_of_relevance_abs")
    # print(pdiffs_rel)
    # print(difference_of_relevance_abs_rel_y)


    print("\nend")

    abs_distance, rel_distance, squared_dist = calculate_distance(X_test, features_abs, features_rel, step)

    print('getting statistics')

    # bo_y_median = np.median(pdiffs_abs)
    # bo_y_stdev = np.std(pdiffs_abs)
    # bor_y_median = np.median(the_difference_of_y)
    # bor_y_stdev = np.std(the_difference_of_y)

    # bo_rel_median = np.median(difference_of_relevance_abs_y)
    # bo_rel_stdev = np.std(difference_of_relevance_abs_y)
    # bor_rel_median = np.median(pdiffs_rel)
    # bor_rel_stdev = np.std(pdiffs_rel)

    if is_global:
        bo_y_mean = np.mean(pdiffs_abs)
        bo_y_stdev = np.std(pdiffs_abs)
        bor_y_mean = np.mean(the_difference_of_y)
        bor_y_stdev = np.std(the_difference_of_y)

        bo_rel_mean = np.mean(difference_of_relevance_abs_y)
        bo_rel_stdev = np.std(difference_of_relevance_abs_y)
        bor_rel_mean = np.mean(pdiffs_rel)
        bor_rel_stdev = np.std(pdiffs_rel)
    else:
        bo_y_mean = ((1 + np.mean(diff_y)) * 100) * (multiplier)
        bo_y_stdev = np.std(diff_y)
        bor_y_mean = ((1 + np.mean(the_difference_of_y)) * 100) * (multiplier)
        bor_y_stdev = np.std(the_difference_of_y)

        bo_rel_mean = ((np.mean(difference_of_relevance_abs_y)) * 100)
        bo_rel_stdev = np.std(difference_of_relevance_abs_y)
        bor_rel_mean = ((np.mean(pdiffs_rel)) * 100)
        bor_rel_stdev = np.std(pdiffs_rel)

        abs_iter, err = get_iterations_till_target(y_test, features_abs, iterations_abs, reg, step, multiplier)
    # print(f"abs erriter for {name} on {model} : {abs_iter}")
        abs_iter = np.mean(abs_iter)


        rel_iter, err = get_iterations_till_target(y_test, features_rel, iterations_rel, reg, step, multiplier)
   #  print(f"rel erriter for {name} on {model} : {rel_iter}")
        rel_iter = np.mean(rel_iter)



    '''
    bo_rel_median = np.median(pdiffs_rel)
    bo_rel_stdev = np.std(pdiffs_rel)
    bor_rel_median = np.median(difference_of_relevance_abs_y)
    bor_rel_stdev = np.std(difference_of_relevance_abs_y)
    '''
    a_dist = np.mean(abs_distance)
    r_dist = np.mean(rel_distance)
    # p_dist = np.mean(percent_dist)
    s_dist = np.mean(squared_dist)

    # return {
    #     "name": name,
    #     "model": model,
    #     "bo_y_median": bo_y_median,
    #     "bo_y_stdev": bo_y_stdev,
    #     "bor_y_median": bor_y_median,
    #     "bor_y_stdev": bor_y_stdev,
    #     "bo_rel_median": bo_rel_median,
    #     "bo_rel_stdev": bo_rel_stdev,
    #     "bor_rel_median": bor_rel_median,
    #     "bor_rel_stdev": bor_rel_stdev,
    #     "percent_dist": p_dist,
    #     "squared_dist": s_dist
    # }

    if is_global:

        return {
            "name": name,
            "model": model,
            "bo_y_mean": bo_y_mean,
            "bo_y_stdev": bo_y_stdev,
            "bor_y_mean": bor_y_mean,
            "bor_y_stdev": bor_y_stdev,
            "bo_rel_mean": bo_rel_mean,
            "bo_rel_stdev": bo_rel_stdev,
            "bor_rel_mean": bor_rel_mean,
            "bor_rel_stdev": bor_rel_stdev,
            "abs_dist": a_dist,
            "rel_dist": r_dist,
            "squared_dist": s_dist
        }

    return {
        "name": name,
        "model": model,
        "bo_y_mean": bo_y_mean,
        "bo_y_stdev": bo_y_stdev,
        "bor_y_mean": bor_y_mean,
        "bor_y_stdev": bor_y_stdev,
        "bo_rel_mean": bo_rel_mean,
        "bo_rel_stdev": bo_rel_stdev,
        "bor_rel_mean": bor_rel_mean,
        "bor_rel_stdev": bor_rel_stdev,
        "abs_iter": abs_iter,
        "rel_iter": rel_iter,
        "abs_dist": a_dist,
        "rel_dist": r_dist,
        "squared_dist": s_dist
    }


def get_all_data(name, model, is_global, multiplier, step):
    '''
    Args:
    - name (str)    : name of dataset
    - model (str)   : name of model
    - is_global (bool)      : if we are optimizing for the maximum we can get or a specific target
    - multiplier (float)    : target multiplier
    - step (int)            : data point frequency

    finds csv files that stored the results and reads data from them + dataset and return it for use
    '''

    X_train, y_train, X_test, y_test, features, ph, phis, reg = pre_process_data(name, model)

    # get paths to csv for us to open
    pdiffs_abs_file, pdiffs_rel_file, f_abs_file, f_rel_file, r_abs_file, r_rel_file = get_paths(name, model, is_global, multiplier)


    # read data from files and store them
    pdiffs_abs = read_file(pdiffs_abs_file)
    pdiffs_rel = read_file(pdiffs_rel_file)
    features_abs = read_file(f_abs_file)
    features_rel = read_file(f_rel_file)
    iterations_abs = read_file(r_abs_file)
    iterations_rel = read_file(r_rel_file)

    # step = round(len(X_test) / len(pdiffs_abs))

    return X_train, y_train, X_test, y_test, ph, phis, reg, pdiffs_abs, pdiffs_rel, features_abs, features_rel, iterations_abs, iterations_rel, step

def test_files(names):
    '''
    Args:
    - names (list[str]) : names of datasets

    Checks if the the datasets valid
    For all dataset files (not including insurance.csv)
    '''
    for name in names:
        if file_is_of_dataset(name): continue
        return False
    return True

def file_is_of_dataset(name):
    '''
    Args:
    - name (str) : name of file to check
    checks if a file is part of the datasets with innate function to load it
    '''
    try: load_dataset(name)
    except Exception: return False
    return True

def convert_results_to_csv(filename, results):
    '''
    Args:
    - filename (str)        : name of file
    - results (dictionary)  : results

    converts the results to a csv so we can store it
    '''
    keys = results[0].keys()

    with open(filename, 'w', newline='') as out:
        dict_writer = csv.DictWriter(out, keys)
        dict_writer.writeheader()
        dict_writer.writerows(results)

def update_csv(fields, file_intermediate, file_result, is_global):
    '''
    Args:
    - fields (list[str])    : csv fields
    - file_in (str)         : read file
    - file_out (str)        : write file

    update csv with new structure
    '''
    with open(file_result, 'w') as out:
        writer = csv.DictWriter(out, fieldnames=fields)
        writer.writeheader()


    # open csv to read from
        f = open(file_intermediate, "r")
        for line in csv.DictReader(f):
            for val in line:
                try: line[val] = float(line[val])
                except: continue

            if is_global:
                # FOR Y OPT
                writer.writerow({
                    'Name': line['name'],
                    'Model': line['model'],
                    'Optimize For': 'Y',
                    'Y Mean': round(float(line['bo_y_mean']), 5),
                    'Y SD': round(line['bo_y_stdev'], 5),
                    'Relevance Mean': round(float(line['bo_rel_mean']), 5),
                    'Relevance SD': round(float(line['bo_rel_stdev']), 5),
                    'Distance': round(line['abs_dist'], 5),
                    'Squared Distance': round(line['squared_dist'], 5),
                })
                # FOR REL OPT
                writer.writerow({
                    'Name': line['name'],
                    'Model': line['model'],
                    'Optimize For': 'Relevance',
                    'Y Mean': round(line['bor_y_mean'], 5),
                    'Y SD': round(line['bor_y_stdev'], 5),
                    'Relevance Mean': round(line['bor_rel_mean'], 5),
                    'Relevance SD': round(line['bor_rel_stdev'], 5),
                    'Distance': round(line['rel_dist'], 5),
                    'Squared Distance': round(line['squared_dist'], 5),
                })

            else:

                # FOR Y OPT
                writer.writerow({
                    'Name': line['name'],
                    'Model': line['model'],
                    'Optimize For': 'Y',
                    'Y Mean': round(float(line['bo_y_mean']), 5),
                    'Y SD': round(line['bo_y_stdev'], 5),
                    'Relevance Mean': round(float(line['bo_rel_mean']), 5),
                    'Relevance SD': round(float(line['bo_rel_stdev']), 5),
                    'Iterations': round(line['abs_iter'], 5),
                    'Distance': round(line['abs_dist'], 5),
                    'Squared Distance': round(line['squared_dist'], 5),
                })
                # FOR REL OPT
                writer.writerow({
                    'Name': line['name'],
                    'Model': line['model'],
                    'Optimize For': 'Relevance',
                    'Y Mean': round(line['bor_y_mean'], 5),
                    'Y SD': round(line['bor_y_stdev'], 5),
                    'Relevance Mean': round(line['bor_rel_mean'], 5),
                    'Relevance SD': round(line['bor_rel_stdev'], 5),
                    'Iterations': round(line['rel_iter'], 5),
                    'Distance': round(line['rel_dist'], 5),
                    'Squared Distance': round(line['squared_dist'], 5),
                })
        f.close()


if __name__ == '__main__':
    # print(run_file('insurance', 'rf'))
    pass

In [ ]:
# must be absolute path to a directory
path = f"/Users/[your user]/"


In [13]:
name = 'insurance'
is_global = False
multiplier = 1.2
step = 100
models = ['rf', 'lgbm']
get_algorithmic_recourse_results(name, models[0], step, is_global, multiplier)

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region'], dtype='object')
received data
age, bmi, sex_female, sex_male, children_0, children_1, children_2, children_3, children_4, children_5, smoker_no, smoker_yes, region_northeast, region_northwest, region_southeast, region_southwest
received phi/phis
received regressor
starting baysopt
start abs bayesopt
['age', 'bmi', 'sex_female', 'sex_male', 'children_0', 'children_1', 'children_2', 'children_3', 'children_4', 'children_5', 'smoker_no', 'smoker_yes', 'region_northeast', 'region_northwest', 'region_southeast', 'region_southwest']
end abs bayesopt
start rel bayesopt
end rel bayesopt


In [12]:
names = ['insurance']
models = ['rf']
is_global = True
multipier = 1.2
step = 100
process_results(names, models, "a", "b", is_global, multiplier, step)


Running insurance.
Running rf.
Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region'], dtype='object')
received data
age, bmi, sex_female, sex_male, children_0, children_1, children_2, children_3, children_4, children_5, smoker_no, smoker_yes, region_northeast, region_northwest, region_southeast, region_southwest
received phi/phis
received regressor
getting diff
pdiffs_abs EQUAL to diff_y

pdiffs_rel EQUAL to difference_of_relevance_abs

end
getting statistics
Finisihed running insurance with rf.
Finished converting results!
 File is named: b
